In [ ]:
!cp -r /kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle /kaggle/efficientnet_kaggle 
! pip install /kaggle/efficientnet_kaggle

# RANZCR - Simple Grad-cam viewer

Based on [Grad-CAM class activation visualization](https://github.com/keras-team/keras-io/blob/master/examples/vision/grad_cam.py).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.cm as cm
import plotly.express as px
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow import keras

import json

import efficientnet.tfkeras # needed by our model

## Model

In [ ]:
model = tf.keras.models.load_model('../input/ranzcr-efn-models/effn_B4_TPU_2.h5')

## Configurable parameters
You can change these to another model.
To get the values for `last_conv_layer_name` and `classifier_layer_names`, use
 `model.summary()` to see the names of all layers in the model.




In [ ]:

img_size = (380, 380)

last_conv_layer_name = "efficientnet-b4"
classifier_layer_names = [
    "global_average_pooling2d",
    "dense",
]


In [ ]:
def make_gradcam_heatmap(
    img_array, model, last_conv_layer_name, classifier_layer_names
):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    #last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        #last_conv_layer_output = last_conv_layer_model(img_array)
        last_conv_layer_output = last_conv_layer(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

def superimpose(image, heatmap):
    # We rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # We use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # We use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # We create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((image.shape[1], image.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * 0.4 + image / 2
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
    
    return(superimposed_img)

In [ ]:
def read_image(file):
    return(tf.image.decode_jpeg(tf.io.read_file(file), channels=3))

def preprocess_image(image):
    image = tf.image.resize(image, img_size)
    return(image / 255.0)


def plot(UID, img, r, annotations, preds):
    fig = px.imshow(img, height=800)
    if annotations.isnull().values.any() == False:
        for i, ann in annotations.iterrows():
            df = pd.DataFrame(json.loads(ann.data),columns=['x', 'y'])
            fig.add_trace(go.Scatter(x=df.x, y=df.y, name=ann.label))
    
    title = 'L: '
    for name, value in r.iteritems():
        if value == 1:
            title += ' ' + name

    title += ' P:'
    for (name, _), pred in zip(r.iteritems(), preds):
        title += f' {name}: {pred:.1f}'


    fig.update_layout(title_text=UID, xaxis_title=title) 
    fig.show()

In [ ]:
df_train = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')
df_train.columns = ['UID','ETTA','ETTB','ETTN','NGTA','NGTB','NGTI','NGTN','CVCA','CVCB','CVCN','SGCP','PID']
df_annotations = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train_annotations.csv')
df_annotations.columns = ['UID','label','data']

def xray(query):
    df = df_train.query(query).sample(1)
    an = df.join(df_annotations.set_index('UID'),how='left',on='UID')
    image = read_image('../input/ranzcr-clip-catheter-line-classification/train/' + df.iloc[0].UID + '.jpg')
    
    # Prepare image
    img_array = np.expand_dims(preprocess_image(image), axis=0)

    # Print what the top predicted class is
    preds = model.predict(img_array)
    
    # Generate class activation heatmap
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names)

    image = superimpose(image, heatmap)
    plot(df.iloc[0].UID, image, df.iloc[0,1:12], an[['label','data']], preds[0])


In [ ]:
# examples:
# xray('CVCA == 1') # -- view a random X-Ray with CVC abnormal
# xray("UID == '1.2.826.0.1.3680043.8.498.59757398491099579448057921213132792160'") # -- view a specific X-Ray by UID
# xray('SGCP == 1') # -- view a random X-Ray with Swan Ganz catheter
# xray('SGCP == 1 and CVCA == 1') # -- view a random X-Ray with Swan Ganz catheter and CVC abnormal

xray("UID == '1.2.826.0.1.3680043.8.498.80711700719709146740499380132484057461'")
